In [1]:
import os
import dspy
import csv
# import pandas as pd

In [4]:
# import ujson
import faiss
from langchain_community.vectorstores import FAISS

In [3]:
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings

<!-- ```mermaid
graph LR;
    A--> B & C & D;
    B--> A & E;
    C--> A & E;
    D--> A & E;
    E--> B & C & D; -->

In [4]:
# #generate llm object
# local_config = {
#     "api_base": "http://localhost:11434/v1",  # 注意需加/v1路徑
#     "api_key": "NULL",  # 特殊標記用於跳過驗證
#     "model": "deepseek-r1:7b",
#     "custom_llm_provider":"deepseek"
# }

# dspy.configure(
#     lm=dspy.LM(
#         **local_config
#     )
# )
# # 測試問答
# qa = dspy.Predict('question -> answer')
# response = qa(question="中國唐朝有幾任皇帝?")
# print(f"模型回答：{response.answer}")


### ref: https://dspy.ai/tutorials/rag/

In [5]:
# def InitializeLLM():
#     local_config = {
#         "api_base": "http://localhost:11434/v1",  # 注意需加/v1路徑
#         "api_key": "NULL",  # 特殊標記用於跳過驗證
#         "model": "deepseek-r1:7b",
#         "custom_llm_provider":"deepseek"
#     }
#     dspy.configure(
#         lm=dspy.LM(
#             **local_config
#         )
#     )

In [5]:
def InitializeLLM():
    lm = dspy.LM('ollama_chat/llama3.2', api_base='http://localhost:11434', api_key='')
    dspy.configure(lm=lm)

In [6]:
InitializeLLM()

In [7]:
def llm_predict(queryStr:str=None):
    if queryStr == None:
        raise ValueError("query string is none, please input query string.")
    promptPatternStr = "question -> answer"
    qa = dspy.Predict(promptPatternStr);
    response = qa(question=queryStr);
    print(f"llm:{response.answer}");

In [8]:
llm_predict("what day is today")

llm:Today is [current date].


In [9]:
dspy.inspect_history(n=1)





[2025-02-24T16:41:15.044820]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.


User message:

[[ ## question ## ]]
what day is today

Respond with the corresponding output fields, starting with the field `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## answer ## ]]
Today is [current date].

[[ ## completed ## ]]







In [13]:
cot = dspy.ChainOfThought('question -> response')
cot(question="should curly braces appear on their own line?")

Prediction(
    reasoning='{The reasoning field is where you would provide a detailed thought process or explanation based on the input question. It should be structured in a way that clearly shows how you arrived at your answer.}',
    response='{The response field is where you provide a concise and direct answer to the question, typically one or two sentences long. It should be clear and to the point, summarizing the key points from your reasoning.}'
)

In [20]:
# with open("./data/ragqa_arena_tech_examples.jsonl") as f:
#     data = [ujson.loads(line) for line in f]

### Basic RAG
- dspy.Embedder: https://dspy.ai/api/models/Embedder/
- multihop Search: https://dspy.ai/tutorials/multihop_search/?h=search

In [9]:
max_characters = 6000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 5  # number of documents to retrieve per search query

with open("../data/ragqa_arena_tech_corpus.jsonl") as f:
    corpus = [ujson.loads(line)['text'][:max_characters] for line in f]
    print(f"Loaded {len(corpus)} documents. Will encode them below.")
model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device="cpu")
embedder = dspy.Embedder(model=model.encode, dimensions=512)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve)

FileNotFoundError: [Errno 2] No such file or directory: '../data/ragqa_arena_tech_corpus.jsonl'

In [34]:
corpus[1]

'You can use StartSound.PrefPane which basically just sets the volume to 0 when you shutdown and then turns it back up after login.'

In [10]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond=dspy.ChainOfThought('context, question -> answer');

    def forward(self,question):
        context = search(question).passages
        return self.respond(context=context, question=question)

In [11]:
rag = RAG();
# print(rag(question="what are high memory and low memory on linux?"));

In [12]:
# dspy.inspect_history()

In [15]:
class RAGChainOfThought(dspy.Module):
    def __init__(self):
        super().__init__()
        
        # Define the chain of thought predictor
        self.qa_chain = dspy.ChainOfThought('context, question -> answer')
        
        # Define retrieval module
        self.retrieve = dspy.Retrieve(k=3)  # Retrieve top 3 relevant passages
    
    def forward(self, question):
        # Retrieve relevant contexts
        retrieved_contexts = self.retrieve(question).passages
        
        # Combine contexts
        context = " ".join(retrieved_contexts)
        
        # Generate answer using chain of thought reasoning
        prediction = self.qa_chain(context=context, question=question)
        
        return prediction.answer

In [12]:
def read_embedding_and_query(idx_path=None, local_db_path=None):
    load_index(idx_path=idx_path);
    load_local_db(local_db_path=local_db_path);

def load_index(idx_path=None):
    #載入預訓練的FAISS索引
    try:
        index = faiss.read_index(idx_path)
        print(f"成功載入FAISS索引，包含 {index.ntotal} 個向量")
        return index
    except Exception as e:
        print(f"索引載入失敗: {str(e)}")
        return None

def load_local_db(local_db_path=None):
    #載入完整的向量資料庫
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    )
    try:
        db = FAISS.load_local(
            folder_path=local_db_path,
            embeddings=embeddings,
            allow_dangerous_deserialization=True  # 必要安全參數
        )
        print(f"載入成功，共 {db.index.ntotal} 筆技術問答")
        return db
    except Exception as e:
        print(f"向量庫載入異常: {str(e)}")
        return None

In [14]:
_idx_path = "./db/qa_index.faiss"
_local_db = "./db/tech_support_faiss/"
read_embedding_and_query(idx_path=_idx_path,local_db_path=_local_db)

成功載入FAISS索引，包含 145 個向量
載入成功，共 145 筆技術問答
